# How to query DFNDB with postgresql in python

### Import python modules (install these if you dont have them)

In [1]:
#External Python Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import importlib

#Add 'functions' folder to notebook directory
cwd = os.getcwd()
fn_dir = cwd + '/functions'
sys.path.append(fn_dir)

#Functions in 'functions' folder
import fn_sql
import fn_doi2bib
import fn_insert
import fn_getfile
import fn_db

### Initiate database connection

In [2]:
db_connection = {
    'address' : 'dfn-parameters.postgres.database.azure.com',
    'port' : '5432',
    'username' : 'testuser@dfn-parameters',
    'password' : 'testuserpassword',
    'dbname' : 'dfndb',
}

db_connection = fn_sql.sqlalchemy_connect(db_connection) #Make connection
dfndb = db_connection['dbobject']

### Query
Pass PostgreSQL queries in string form as QUERY

For example, here we list all (*) properties from the 'paper' table, showing all entries in the table:

In [3]:
QUERY = '''
        SELECT *
        FROM paper;
        '''
df = pd.read_sql(QUERY,dfndb)
df

,paper_id,doi,paper_tag,year,title,authors,url,created_on,accepted
0,3,10.1149/2.0571912jes,Landesfeind2019,2019,Temperature and Concentration Dependence of th...,"Johannes Landesfeind, Hubert A. Gasteiger",https://doi.org/10.1149%2F2.0571912jes,2020-09-14 23:06:25.477299+00:00,True
1,1,10.1149/2.0551509jes,Ecker2015,2015,Parameterization of a Physico-Chemical Model o...,"Madeleine Ecker, Thi Kim Dung Tran, Philipp De...",https://doi.org/10.1149%2F2.0551509jes,2020-09-02 22:08:23.534717+00:00,True
2,2,10.1149/2.0321816jes,Schmalstieg2018,2018,Full Cell Parameterization of a High-Power Lit...,"Johannes Schmalstieg, Christiane Rahe, Madelei...",https://doi.org/10.1149%2F2.0321816jes,2020-09-14 10:01:27.983468+00:00,True


### What are the parameters reported for a given paper?
List all parameters and  material it applies to based on a given 'paper_tag'


In [4]:
QUERY = '''
        SELECT paper.paper_tag, parameter.name, parameter.symbol,material.class AS "Material Class", material.name AS "Material Name"
        FROM paper 
        JOIN data ON data.paper_id = paper.paper_id
        JOIN parameter ON parameter.parameter_id = data.parameter_id
        JOIN material ON material.material_id = data.material_id
        WHERE paper.paper_tag = 'Ecker2015';
        '''
df = pd.read_sql(QUERY,dfndb)
df

,paper_tag,name,symbol,Material Class,Material Name
0,Ecker2015,Maxmimum Concentration,$c_\text{a}^\text{max}$,anode,graphite
1,Ecker2015,Porosity,$\varepsilon_\text{a}$,anode,graphite
2,Ecker2015,Diffusion Coefficient,$D_\text{e}$,electrolyte,LiPF6:EC:EMC 1:1
3,Ecker2015,Maximum Concentration,$c_\text{c}^\text{max}$,cathode,NCO46
4,Ecker2015,Ionic Conductivity,$\kappa$,electrolyte,LiPF6:EC:EMC 1:1
5,Ecker2015,Tortuosity,$\tau_\text{a}$,anode,graphite
6,Ecker2015,Porosity,$\varepsilon_\text{c}$,cathode,NCO46
7,Ecker2015,Diffusion Coefficient,$D_\text{c}$,cathode,NCO46
8,Ecker2015,Diffusion Coefficient,$D_\text{a}$,anode,graphite
9,Ecker2015,Electronic Conductivity,$\sigma_\text{a}$,anode,graphite


### What electrolyte formulations have parameters?

In [43]:
QUERY = '''
        SELECT DISTINCT material.name,material.lipf6, material.ec, material.emc, material.dmc, material.fec, material.dec, paper.paper_tag
        FROM material
        JOIN data ON data.material_id = material.material_id
        JOIN paper on paper.paper_id = data.paper_id
        WHERE material.class = 'electrolyte';
        '''
df = pd.read_sql(QUERY,dfndb)
df

,name,lipf6,ec,emc,dmc,fec,dec,paper_tag
0,LiPF6 in EC:DMC:EMC 1:1:1,1.0,0.333333,0.333333,0.333333,0.00,0.0,Schmalstieg2018
1,LiPF6:EC:DMC 1:1,1.0,0.500000,0.000000,0.500000,0.00,0.0,Landesfeind2019
2,LiPF6:EC:EMC 1:1,1.0,0.500000,0.500000,0.000000,0.00,0.0,Ecker2015
3,LiPF6:EC:EMC 3:7,1.0,0.300000,0.700000,0.000000,0.00,0.0,Landesfeind2019
4,LiPF6:EMC:FEC 19:1,1.0,0.000000,0.950000,0.000000,0.05,0.0,Landesfeind2019


### What Parameters are measured with GITT?

In [44]:
QUERY = '''
        SELECT DISTINCT method.name, parameter.name ,parameter.symbol, data.raw_data_class, paper.paper_tag ,material.name ,material.class
        FROM data
        JOIN paper ON data.paper_id = paper.paper_id
        JOIN material ON material.material_id = data.material_id
        JOIN data_method ON data_method.data_id = data.data_id
        JOIN parameter ON parameter.parameter_id = data.parameter_id
        JOIN method ON method.method_id = data_method.method_id
        WHERE method.name = 'GITT';
        '''
df = pd.read_sql(QUERY,dfndb)
df

,name,name,symbol,raw_data_class,paper_tag,name,class
0,GITT,Diffusion Coefficient,$D_\text{a}$,array,Ecker2015,graphite,anode
1,GITT,Diffusion Coefficient,$D_\text{a}$,array,Schmalstieg2018,graphite,anode
2,GITT,Diffusion Coefficient,$D_\text{c}$,array,Ecker2015,NCO46,cathode
3,GITT,Diffusion Coefficient,$D_\text{c}$,array,Schmalstieg2018,NMC111,cathode
4,GITT,Half Cell OCV,"$U_\text{eq,a}$",array,Ecker2015,graphite,anode
5,GITT,Half Cell OCV,"$U_\text{eq,a}$",array,Schmalstieg2018,graphite,anode
6,GITT,Half Cell OCV,"$U_\text{eq,c}$",array,Ecker2015,NCO46,cathode
7,GITT,Half Cell OCV,"$U_\text{eq,c}$",array,Schmalstieg2018,NMC111,cathode


### Which solid diffusivities are reported for graphite at room temperature?

In [45]:
QUERY = '''
        SELECT parameter.name, material.name, paper.paper_tag, data.temp_range, method.name, data.function
        FROM data
        JOIN paper ON paper.paper_id = data.paper_id
        JOIN material ON material.material_id = data.material_id
        JOIN parameter ON parameter.parameter_id = data.parameter_id
        JOIN data_method ON data_method.data_id = data.data_id
        JOIN method ON method.method_id = data_method.method_id
        WHERE parameter.name = 'Diffusion Coefficient'
        AND
        material.name ='graphite'
        AND 
        298 BETWEEN lower(data.temp_range) AND upper(data.temp_range);
        '''
df = pd.read_sql(QUERY,dfndb)
df

,name,name,paper_tag,temp_range,name,function
0,Diffusion Coefficient,graphite,Ecker2015,"[250, 333]",GITT,"[b'd', b'e', b'f', b' ', b'f', b'u', b'n', b'c..."
1,Diffusion Coefficient,graphite,Ecker2015,"[250, 333]",EIS,"[b'd', b'e', b'f', b' ', b'f', b'u', b'n', b'c..."
2,Diffusion Coefficient,graphite,Schmalstieg2018,"[250, 313]",GITT,"[b'd', b'e', b'f', b' ', b'f', b'u', b'n', b'c..."
